In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints

import nest_asyncio
nest_asyncio.apply()

PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

sys.path.append("../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_jitted_funs
from python_helpers import helper_get_alpha

from python_helpers.data_figures import data_phase_equilibria_solid_lr
from python_helpers.data_figures import data_critical_and_triple_point_feanneos, data_critical_and_triple_point_feanneos_by_parts


In [2]:
##########################
# Folder to save results #
##########################

folder_to_save = '../computed_files'
os.makedirs(folder_to_save, exist_ok=True)

plot_figure = False # already checked that it converged nicely
save_files = False

In [3]:
#########################
# Loading FE-ANN(s) EoS #
#########################

ckpt_folder = '../ann_models/feanns_eos'
prefix_params = 'FE-ANN-EoS-params_'
###
Tscale = 'Tinv'
seed = 17
factor = 0.01
EPOCHS = 50000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params, step=EPOCHS)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}

fun_dic = helper_jitted_funs(helmholtz_model, helmholtz_params)

In [4]:
lr_min = 7.
lr_max = 34.
n_min = 25
n_max = 25
lr0 = 12
inc0_triple = [1e-3, 0.85, 0.95, 0.67]
df_crit_triple_feann = data_critical_and_triple_point_feanneos_by_parts(fun_dic, lr_min, lr_max, n_min=n_min, n_max=n_max, lr0=lr0, inc0_triple=inc0_triple)
df_crit_triple_feann.reset_index(drop=True, inplace=True)
interp1d_crit = interp1d(df_crit_triple_feann['lr'], df_crit_triple_feann[['rhoad_crit','Tad_crit']].T, kind='cubic')
interp1d_triple = interp1d(df_crit_triple_feann['lr'], df_crit_triple_feann[['rhovad_triple','rholad_triple', 'rhosad_triple', 'Tad_triple']].T, kind='cubic')

df_crit_triple_feann.head()

2025-03-25 13:12:47.498031: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-25 13:12:47.580289: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-25 13:12:47.603593: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-25 13:12:47.611258: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


,lr,la,alpha,Tad_crit,rhoad_crit,Pad_crit,success_crit,Tad_triple,rhovad_triple,rholad_triple,rhosad_triple,Pad_triple,success_triple,Tc/Tt
0,7.000000,6.0,1.470949,1.959135,0.278703,0.177040,True,0.750193,0.000101,0.914291,0.987359,0.000076,True,2.611507
1,7.208333,6.0,1.419953,1.908827,0.282007,0.174709,True,0.745504,0.000129,0.907169,0.983010,0.000096,True,2.560451
2,7.416667,6.0,1.373673,1.862984,0.284981,0.172690,True,0.740155,0.000160,0.900707,0.979025,0.000118,True,2.517018
3,7.625000,6.0,1.331478,1.820779,0.287626,0.170846,True,0.734467,0.000191,0.894887,0.975414,0.000140,True,2.479048
4,7.833333,6.0,1.292841,1.781612,0.289973,0.169087,True,0.728688,0.000223,0.889670,0.972180,0.000162,True,2.444959


In [5]:
n = 50
lr_array = np.linspace(lr_min, lr_max, n)
Tlower = 0.4
Tupper = 10.
n_vle = 100
n_sle = 1000
n_sve = 50
df_info_list = []
df_vle_list = []
df_sle_list = []
df_sve_list = []

i = 0
for i in range(n):
    lambda_r = lr_array[i]
    initial_crit_point = interp1d_crit(lambda_r)
    initial_triple_point = interp1d_triple(lambda_r)
    phase_equilibria_lr = data_phase_equilibria_solid_lr(fun_dic, lambda_r=lambda_r, 
                                                        initial_crit_point=initial_crit_point, 
                                                        initial_triple_point=initial_triple_point,
                                                        n_vle=n_vle, n_sle=n_sle, n_sve=n_sve,
                                                        Tlower=Tlower, Tupper=Tupper)
    df_info = phase_equilibria_lr['info']
    df_vle = phase_equilibria_lr['vle']
    df_sle = phase_equilibria_lr['sle']
    df_sve = phase_equilibria_lr['sve']
    for df in [df_vle, df_sle, df_sve]:
        df['lambda_r'] = 1. * lambda_r
        df['lambda_a'] = 6.
        df['alpha'] = helper_get_alpha(df['lambda_r'], df['lambda_a'])
    df_vle['Tr_vle_model'] = df_vle['T_vle_model'].to_numpy() / df_info['Tcad_model'].to_numpy()

    df_info_list.append(df_info)
    df_vle_list.append(df_vle)
    df_sle_list.append(df_sle)
    df_sve_list.append(df_sve)

    if plot_figure:
        fig = plt.figure(figsize=(10, 3))
        ax1 = fig.add_subplot(131)
        ax2 = fig.add_subplot(132)
        ax3 = fig.add_subplot(133)

        ax1.plot(df_vle['rhov_vle_model'], df_vle['T_vle_model'], color='k')
        ax1.plot(df_vle['rhol_vle_model'], df_vle['T_vle_model'], color='k')

        ax1.plot(df_sle['rhol_sle_model'], df_sle['T_sle_model'], color='b')
        ax1.plot(df_sle['rhos_sle_model'], df_sle['T_sle_model'], color='b')

        ax1.plot(df_sve['rhov_sve_model'], df_sve['T_sve_model'], color='r')
        ax1.plot(df_sve['rhos_sve_model'], df_sve['T_sve_model'], color='r')

        ax2.semilogy(1/df_vle['T_vle_model'], df_vle['P_vle_model'], color='k')
        ax2.semilogy(1/df_sle['T_sle_model'], df_sle['P_sle_model'], color='b')
        ax2.semilogy(1/df_sve['T_sve_model'], df_sve['P_sve_model'], color='r')

        ax3.plot(df_vle['T_vle_model'], df_vle['Uvap_vle_model'], color='k')
        ax3.plot(df_sle['T_sle_model'], df_sle['Umelting_sle_model'], color='b')
        ax3.plot(df_sve['T_sve_model'], df_sve['Usub_sve_model'], color='r')

        fig.suptitle(f'lambda_r = {lambda_r:.2f}')


2025-03-25 13:12:49.820198: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-25 13:12:50.068453: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


In [6]:
df_info_all = pd.concat(df_info_list).reset_index(drop=True)
df_vle_all = pd.concat(df_vle_list).reset_index(drop=True)
df_sle_all = pd.concat(df_sle_list).reset_index(drop=True)
df_sve_all = pd.concat(df_sve_list).reset_index(drop=True)
if save_files:
    filename = 'phase_equilibria_solid.xlsx'
    file_to_save = os.path.join(folder_to_save, filename)
    writer = pd.ExcelWriter(file_to_save, engine='xlsxwriter')
    df_info_all.to_excel(writer, sheet_name='info', index=False)
    df_vle_all.to_excel(writer, sheet_name='vle', index=False)
    df_sle_all.to_excel(writer, sheet_name='sle', index=False)
    df_sve_all.to_excel(writer, sheet_name='sve', index=False)
    writer.close()
    print(f'File saved in {file_to_save}')